## 환경변수 로딩

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

## Pinecone 객체생성

In [4]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
pc.list_indexes()

[
    {
        "name": "quickstart",
        "metric": "cosine",
        "host": "quickstart-voqt390.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 1024,
        "deletion_protection": "disabled",
        "tags": null
    },
    {
        "name": "wiki",
        "metric": "cosine",
        "host": "wiki-voqt390.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 1536,
        "deletion_protection": "disabled",
        "tags": null
    }
]

## index 객체 생성


In [5]:
# 사용할 인덱스 이름을 지정합니다.
index_name = "wiki"

# 지정한 이름으로 Pinecone 인덱스를 가져옵니다.
index = pc.Index(index_name)

# 인덱스의 통계 정보를 설명합니다.
index.describe_index_stats()

c:\Users\Admin\miniconda3\envs\lc_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'wiki_ns1': {'vector_count': 1500}},
 'total_vector_count': 1500,
 'vector_type': 'dense'}

## 검색하기 (Pinecone)

In [6]:
from langchain_openai import OpenAIEmbeddings

question = ["벨기에(Belgium)는 어디 있나요?"]

embedding = OpenAIEmbeddings(model="text-embedding-3-small", api_key=OPENAI_API_KEY)

embedded_question = embedding.embed_documents(question)

print(embedded_question)

[[-0.03147738426923752, 0.021170910447835922, 0.0714610368013382, 0.027771687135100365, -0.06320743262767792, -0.023644885048270226, -0.01403322909027338, 0.04215232655405998, -0.030277244746685028, -0.03490936756134033, -0.02360277622938156, 0.022297358140349388, -0.008316767401993275, -0.04728977009654045, -0.010832852683961391, 0.03743598237633705, -0.04973216354846954, 0.03438299149274826, 0.010506498627364635, 0.0011705324286594987, -0.027161089703440666, -0.019244369119405746, -0.04623701795935631, 0.03318284824490547, 0.04408939555287361, 0.00933267641812563, 0.010053813457489014, -0.04305769503116608, 0.004926895257085562, -0.038951948285102844, 0.021897312253713608, -0.037941303104162216, -0.0008757608593441546, -0.021065635606646538, -0.00022864530910737813, -0.004711080342531204, 0.014875433407723904, -0.05276409909129143, 0.03333023563027382, 0.01184349786490202, -0.029287654906511307, -0.026529435068368912, 0.024234429001808167, 0.02806645818054676, -0.01783367618918419, 0

In [13]:
query_result = index.query(
    namespace="wiki_ns1",
    vector=embedded_question,
    top_k=5,
    include_vector=False,
    include_metadata=True
)

print(query_result)

result_ids = [r.id for r in query_result.matches]

print(result_ids)

{'matches': [{'id': 'e15fad33-c735-4dc9-8466-6666011e893a',
              'metadata': {'chunk_id': 0.0,
                           'full_text': 'Belgium, officially the Kingdom of '
                                        'Belgium (Dutch: Koninkrijk België, '
                                        'German: Königreich Belgien, French: '
                                        'Royaume de Belgique), is a federal '
                                        'state in Western Europe. Belgium has '
                                        'an area of . Around 11 million people '
                                        'live in Belgium. It is a founding '
                                        'member of the European Union and is '
                                        'home to its headquarters. The capital '
                                        'city of Belgium is Brussels, where '
                                        'the European Union, NATO and other '
                             

In [14]:
for r in query_result.matches:
    print(r.score, r.metadata)

0.547039807 {'chunk_id': 0.0, 'full_text': 'Belgium, officially the Kingdom of Belgium (Dutch: Koninkrijk België, German: Königreich Belgien, French: Royaume de Belgique), is a federal state in Western Europe. Belgium has an area of . Around 11 million people live in Belgium. It is a founding member of the European Union and is home to its headquarters. The capital city of Belgium is Brussels, where the European Union, NATO and other famous organisations are based.\n\nThere are three regions in Belgium:\n Flanders is the name of the northern half of Belgium, just south of the Netherlands. Most of the people in this region, called the Flemish people, speak Dutch.\n Wallonia is the name of the southern half of Belgium, just north of France. Here, most of the people, the Walloons, speak French. There is a small part of Wallonia next to the border with Germany where the people speak German.\n The Brussels-Capital Region, where the capital of Brussels is found, is in the middle of the count

## 검색하기 (Langchain)

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index,        
                                    embedding=embedding, 
                                    text_key="full_text")

docs = vector_store.similarity_search(query=question[0], k=5, namespace="wiki_ns1")
print(docs)

for doc in docs:
    print(doc.metadata)
    print(doc.page_content)

[Document(id='e15fad33-c735-4dc9-8466-6666011e893a', metadata={'chunk_id': 0.0, 'title': 'Belgium', 'url': 'https://simple.wikipedia.org/wiki/Belgium', 'wiki_id': '103'}, page_content='Belgium, officially the Kingdom of Belgium (Dutch: Koninkrijk België, German: Königreich Belgien, French: Royaume de Belgique), is a federal state in Western Europe. Belgium has an area of . Around 11 million people live in Belgium. It is a founding member of the European Union and is home to its headquarters. The capital city of Belgium is Brussels, where the European Union, NATO and other famous organisations are based.\n\nThere are three regions in Belgium:\n Flanders is the name of the northern half of Belgium, just south of the Netherlands. Most of the people in this region, called the Flemish people, speak Dutch.\n Wallonia is the name of the southern half of Belgium, just north of France. Here, most of the people, the Walloons, speak French. There is a small part of Wallonia next to the border wit